In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

################################################################################
#
# Example script to process MATISSE data of beta Pic b with GPAO
# Author: fmillour
# Date: 18/11/2024
# Project: OPTRA
# 
################################################################################

from op_corrflux   import *
from op_rawdata    import *
from op_flux       import *
from op_vis        import *
from op_oifits     import *
import numpy as np
import matplotlib.pyplot as plt
from astropy.io    import fits
import os
from scipy.ndimage import median_filter
from scipy         import *
from scipy         import stats

#plt.ion()
plot = 1
plotFringes = plot
plotPhi     = plot
plotDsp     = plot
plotRaw     = plot
plotCorr    = plot
plotPist    = plot

verbose = False

def do_nothing():
    pass

#bbasedir = '/Users/jscigliuto/Nextcloud/DATA/betaPicb/'
#basedir = bbasedir+'betaPicb_rawdata_2024-11-17/'
bbasedir = '/Users/jscigliuto/Desktop/PDS70/'
#bbasedir = os.path.expanduser('~/Documents/ExoMATISSE/beta_Pic_b/')
basedir  = bbasedir

starfiles = os.listdir(basedir)
print(starfiles)
fitsfiles = [f for f in starfiles if ".fits" in f and not "M." in f]
print(fitsfiles)

# select only fits files that correspond to observations
obsfilesL     = []
obsfilesL_MJD = []
obsfilesN     = []
skyfilesL     = []
skyfilesL_MJD = []
darkfiles     = []

for fi in fitsfiles:
    #print(fi)
    fh = fits.open(basedir+fi)
    #op_print_fits_header(fh)
    hdr = fh[0].header
    fh.close()
    inst = hdr['INSTRUME']
    catg = hdr['ESO DPR CATG']
    type = hdr['ESO DPR TYPE']
    try:
        chip = hdr['ESO DET CHIP NAME']
    except:
        do_nothing()
    try:
        dit  = hdr['ESO DET SEQ1 DIT']
    except:
        do_nothing()
        #print('No DIT in header')
    try:
        ndit = hdr['ESO DET NDIT']
    except:
        do_nothing()
    #print(fi, inst, catg, type, chip, dit, ndit)
    if catg == 'SCIENCE' and type == 'OBJECT' and chip == 'HAWAII-2RG' :
        #print("science file!")
        obsfilesL.append(fi)
        obsfilesL_MJD.append(hdr['MJD-OBS'])
    if catg == 'CALIB' and type == 'STD' and chip == 'HAWAII-2RG':
        #print("calibrator file!")
        obsfilesL.append(fi)
        obsfilesL_MJD.append(hdr['MJD-OBS'])
    if catg == 'CALIB' and type == 'SKY' and chip == 'HAWAII-2RG' :
        #print("sky file!")
        skyfilesL.append(fi)
        skyfilesL_MJD.append(hdr['MJD-OBS'])

skyfilesL_MJD = np.array(skyfilesL_MJD)
starfiles = sorted(obsfilesL)
starfiles = [f for f in starfiles if 'STD' in f]
print('Starfiles:', starfiles)
print('Skyfiles:', skyfilesL)


for ifile in starfiles:
    starfile = basedir + ifile

    fh = fits.open(starfile)
    hdr = fh[0].header
    fh.close()
    mjd_obs = hdr['MJD-OBS']
    
    # associate the two sky files matching properties of the star file
    mjdiff = np.abs(skyfilesL_MJD - mjd_obs)
    # Sort skyfiles by ascending distance to the starfile
    skyfilesL_sorted = [x for _,x in sorted(zip(mjdiff,skyfilesL))]
    
    for isky in skyfilesL_sorted:
        skyfile = basedir + isky
        fh = fits.open(skyfile)
        hdrsky = fh[0].header
        fh.close()
        
        keys_to_match = ['INSTRUME','ESO DET CHIP NAME','ESO DET SEQ1 DIT']
        imatch = 0
        for key in keys_to_match:
            if hdr[key] == hdrsky[key]:
                print('Matching key:', key)
                imatch += 1
        if imatch == len(keys_to_match):
            print('Matching sky file:', skyfile)
            break

    #caldir    = '/Users/jscigliuto/Nextcloud/DATA/CALIB2024/'
    caldir    = '/Users/jscigliuto/Nextcloud/DATA/CALIB2024/'
    kappafile = caldir+'KAPPA_MATRIX_L_MED.fits'
    shiftfile = caldir+'SHIFT_L_MED.fits'
    flatfile  = caldir+'FLATFIELD_L_SLOW.fits'
    badfile   = caldir+'BADPIX_L_SLOW.fits'

    ##########################################################

    bdata = op_loadAndCal_rawdata(starfile, skyfile, badfile, flatfile, verbose=verbose, plot=plotRaw)
    

    ##########################################################

    print('Shape of bdata:', bdata['INTERF']['data'].shape)

    if plotFringes:
        fig, ax1 = plt.subplots(1, 1, figsize=(8, 4), sharex=True, sharey=True)
        # Plot the first frame of intf
        ax1.imshow(np.mean(bdata['INTERF']['data'], axis=0), cmap='viridis')
        ax1.set_title('average intf')

        plt.show()
        
    cfdata = op_get_corrflux(bdata, shiftfile, plot=plotCorr, verbose=verbose)

    wlen = cfdata['OI_WAVELENGTH']['EFF_WAVE']
    #print(wlen)

    #########################################################

    basename = os.path.basename(starfile)
    basen    = os.path.splitext(basename)[0]
    directory = cfdata['hdr']['DATE-OBS'].split('T')[0]+'_OIFITS/'
    if not os.path.exists(bbasedir+directory):
        os.makedirs(bbasedir+directory)
    chip = cfdata['hdr']['ESO DET CHIP NAME']
    if 'HAWAII' in chip:
        band = 'L'
    elif 'AQUARIUS' in chip:
        band = 'N'
    basen = directory+cfdata['hdr']['INSTRUME'][0:3]    + '_' +\
    cfdata['hdr']['DATE-OBS'].replace(':','-')          + '_' +\
    cfdata['hdr']['ESO OBS TARG NAME'].replace(' ','_') + '_' +\
    cfdata['hdr']['ESO DPR CATG'][0:3]                  + '_' +\
    band                                                + '_' +\
    cfdata['hdr']['ESO INS DIL ID']                     + '_' +\
    cfdata['hdr']['ESO INS BCD1 ID']                          +\
    cfdata['hdr']['ESO INS BCD2 ID']
        
    if 0:
        vis2, mask = op_extract_simplevis2(cfdata, verbose=verbose, plot=False)
        #print(mask)
        #print(~mask)
        notvis2 = np.ma.masked_array(np.ma.getdata(vis2), ~mask)
        allvis2 = np.ma.getdata(vis2)
        
        #print('Shape of vis2:', vis2.shape)
        #print('Shape of notvis2:', notvis2.shape)

        fig0, ax0 = plt.subplots(7, 1, figsize=(8, 8), sharex=1, sharey=1)
        #print('Shape of ax1:', ax0.shape)
        colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A1', '#A133FF', '#33FFF5', '#F5FF33']
        for i in np.arange(7):
            #print('i:', i)
            ax0[i].plot(wlen, allvis2[i,:], color='lightgray')
            ax0[i].plot(wlen, vis2[i,:], color=colors[i])
            ax0[i].set_ylabel(f'vis2 {i}')

        #print('Basename of starfile:', basen)
        plt.suptitle(r'Visibility as a function of $\lambda$, {basen}')
        plt.xlim(np.min(wlen), np.max(wlen))
        plt.ylim(-0.1, 1.1)
        #print(os.path.expanduser(bbasedir+f'{basen}_vis2.png'))
        plt.savefig(os.path.expanduser(bbasedir+f'{basen}_vis2.png'))
        #plt.show()

    #########################################################

    #cfdem = op_demodulate(cfdata, wlen, verbose=True, plot=False)
    cfdem = cfdata



    #print('Shape of cfdata:', cfdem['CF']['CF_demod'].shape)
    #cf = cfdem['CF']['CF_achr_phase_corr']
    cf   = cfdem['CF']['CF_Binned']
    wlen = cfdata['OI_WAVELENGTH']['EFF_WAVE_Binned']
    #cf = cfdem['CF']['CF_demod']
    #cf = cfdem['CF']['CF_reord']
    sumcf = np.sum(cf, axis=1)
    #print('Shape of sumcf:', sumcf.shape)
    shp = sumcf.shape
    nbs = shp[0]
    
    if plotCorr:
        fig1, ax1 = plt.subplots(nbs, 2, figsize=(8, 8), sharex=1, sharey=0)
        #print('Shape of ax1:', ax1.shape)
        colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A1', '#A133FF', '#33FFF5', '#F5FF33']
        for i in np.arange(nbs):
            #print('i:', i)
            ax1[i,0].plot(wlen,   np.abs(sumcf[i,:]), color=colors[i])
            if i == 0 and nbs == 7:
                ax1[i,0].set_ylabel(f'flux {i+1}')
            else:
                ax1[i,0].set_ylabel(f'corr. flux {i+1}')
            ax1[i,1].plot(wlen, np.angle(sumcf[i,:]), color=colors[i])
            ax1[i,1].set_ylabel(f'phase {i+1}')
        plt.suptitle('Sum CF data (1 exposure)')
        plt.tight_layout()
        plt.savefig(os.path.expanduser(bbasedir+f'{basen}_corrflux.png'))
        #plt.show()
    
    

    # iframe = 0
    # fig2, ax2 = plt.subplots(2, 6, figsize=(8, 4))
    # for i in np.arange(6): #+1 ????
    #     ax2[0,i].plot(wlen, np.abs(cf[i,iframe,:]), color=colors[i])
    #     ax2[1,i].plot(wlen, np.angle(cf[i,iframe,:]), color=colors[i])
    # plt.title(f'frame {iframe} of CF data')
    # plt.show()

    data, OPD_list = op_get_piston_fft(cfdem, verbose=True, plot=plotPist)
    #print('OPD:',OPD_list)

    #data, slopes = op_get_piston_slope(cfdem, verbose=True, plot=True)
    # print('Slopes:',slopes)

    #data, pistons = op_get_piston_chi2(data, 'fft', verbose=False, plot=True)
    # print('Pistons:',pistons)

    data = op_corr_piston(data, verbose=False, plot=plotPist)

    if plotPist:
        colors = ['#FF5733', '#33FF57', '#3357FF', '#FF33A1', '#A133FF', '#33FFF5', '#F5FF33']
        fig, ax = plt.subplots(7, 2, figsize=(8, 8))
        fig.suptitle('Piston corrected phase')
        for i_base in range(7):
            for i_frame in range(6):
                ax[i_base,0].plot(wlen, np.angle(data['CF']['CF_Binned'][i_base, i_frame]), color=colors[i_base])
                ax[i_base,1].plot(wlen, np.angle(data['CF']['CF_piston_corr'][i_base, i_frame]), color=colors[i_base])
        plt.show()
    

    #########################################################
    outfilename = os.path.expanduser(bbasedir+f'{basen}_corrflux_oi.fits')
    hdr = cfdem['hdr']
    oiwavelength = op_gen_oiwavelength(cfdem, verbose=verbose)
    oitarget     = op_gen_oitarget(cfdem, verbose=True, plot=False)
    oirray       = op_gen_oiarray(cfdem, verbose=True, plot=False)
    oivis        = op_gen_oivis(cfdem, cfin='CF_piston_corr', verbose=verbose, plot=False)
    op_write_oifits(outfilename, hdr, oiwavelength, oirray, oitarget, oivis, oivis2=None, oit3=None)


    '''
    #scfdata = op_sortout_peaks(cfdata, verbose=True)
    #scfdata = cfdata

    iframe = 0
    import matplotlib.animation as animation
    fig, ax = plt.subplots()
    #lines = [ax.plot([], [], color=colors[i])[0] for i in range(1)]
    lines  = [ax.plot([], [], color=colors[i])[0] for i in np.arange(7)]
    #lines2 = [ax.plot([], [], '--', color=colors[i])[0] for i in np.arange(7)]
    #ax.set_xlim(0, cfdata['CF']['CF'].shape[2])
    ax.set_xlim(np.min(wlen), np.max(wlen))
    ax.set_ylim(-np.pi, np.pi)
    ax.set_title('Phase as a function of the wavelength for CF Data')
    def init():
        for line in lines:
            line.set_data([], [])
        return lines
    def update(frame):
        for i, line in enumerate(lines):
            if i == 5:
                #line.set_data(wlen, np.angle(cfdata['CF']['CF'][i, frame, :]  * np.conjugate(cfdata['CF']['mod_phasor'][2, frame, :])))
                
                # CF 1 phi 5 -> 6
                # CF 2 phi 0 -> 1
                # CF 3 phi 3 -> 4
                # CF 4 phi 4 -> 5
                # CF 5 phi 1 -> 2
                # CF 6 phi 2 -> 3
                line.set_data(wlen, np.angle(cfdata['CF']['CF_demod'][i, frame, :]))
                
                #lines2[i].set_data(wlen, np.angle(cfdata['CF']['mod_phasor'][i-1, frame, :]))
                
        return lines
    ani = animation.FuncAnimation(fig, update, frames=cfdata['CF']['CF'].shape[1], init_func=init, blit=True)
    plt.show()
    if plotDsp:
        plt.figure(5)
        for i in np.arange(6)+1:
            if i == 5:
                plt.plot(np.abs(cfdata['CF']['CF'][i,iframe,:]) / np.abs(cfdata['CF']['CF'][0,0,:])*3,color=colors[i])
                plt.plot(np.max(np.abs(cfdata['CF']['data'][i,iframe,:,:]),axis=1) / np.abs(cfdata['CF']['CF'][0,0,:])*3*7,':',color=colors[i])
        plt.ylim(-0.2,1.2)
        plt.title('This one should resemble a visibility curve')
    plt.show()
    '''

['MATISSE_OBS_SIPHOT_LM_OBJECT_077_0025.fits', 'MATISSE_OBS_SIPHOT_LM_SKY_077_0014.fits', 'MATISSE_OBS_SIPHOT_LM_SKY_077_0002.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0033.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0002.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0014.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0009.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0029.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0022.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0013.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0005.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0018.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0019.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0004.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0012.fits', '.DS_Store', 'MATISSE_OBS_SIPHOT_LM_STD_077_0023.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0028.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0008.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0015.fits', 'MATISSE_OBS_SIPHOT_LM_STD_077_0003.fits', 'MATISSE_OBS_SIPHOT_LM_SKY_077_0003.fits', 'MATISSE_OBS_SIPHOT_LM_OBJECT_077_0032.fits', 'MATISS

/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: Mean of empty slice.
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: invalid value encountered in scalar divide
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)


cf shape: (7, 272, 118)
n_base: 7 n_frame: 272
Calculating piston from FFT...
cf shape: (7, 272, 23)
n_base: 7 n_frame: 272
Generating OI_WAVELENGTH...
Shape of EFF_WAVE: (118,)
Generating OI_TARGET...
Generating OI_ARRAY...
Generating OI_VIS...
Shape of complexvis: (6, 272, 23)
Shape of target_IDxxx: (1632,)
Writing OI fits...
Matching key: INSTRUME
Matching key: ESO DET CHIP NAME
Matching key: ESO DET SEQ1 DIT
Matching sky file: /Users/jscigliuto/Desktop/PDS70/MATISSE_OBS_SIPHOT_LM_SKY_077_0002.fits
loading and calibrating raw data...
Loading raw data...
Loading raw data...
Shape of bdata: (272, 118, 625)
Computing FFT of interferograms...
Shape of sum_dsp_intf: (118, 625)
Instrument {'name': 'MATISSE_L', 'ntel': 4, 'recombination': 'multiaxial_allinone', 'interfringe': 83.58899999999998, 'peakwd': 0.9, 'scrP': [1, 2, 4, 3], 'scrB': [[2, 3], [0, 1], [1, 2], [1, 3], [0, 2], [0, 3]], 'bcd_offset': [1, 2, 3, 4, 5, 6], 'ron': 20}


/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: Mean of empty slice.
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: invalid value encountered in scalar divide
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)


cf shape: (7, 272, 118)
n_base: 7 n_frame: 272
Calculating piston from FFT...
cf shape: (7, 272, 23)
n_base: 7 n_frame: 272
Generating OI_WAVELENGTH...
Shape of EFF_WAVE: (118,)
Generating OI_TARGET...
Generating OI_ARRAY...
Generating OI_VIS...
Shape of complexvis: (6, 272, 23)
Shape of target_IDxxx: (1632,)
Writing OI fits...
Matching key: INSTRUME
Matching key: ESO DET CHIP NAME
Matching key: ESO DET SEQ1 DIT
Matching sky file: /Users/jscigliuto/Desktop/PDS70/MATISSE_OBS_SIPHOT_LM_SKY_077_0002.fits
loading and calibrating raw data...
Loading raw data...
Loading raw data...
Shape of bdata: (272, 118, 625)
Computing FFT of interferograms...
Shape of sum_dsp_intf: (118, 625)
Instrument {'name': 'MATISSE_L', 'ntel': 4, 'recombination': 'multiaxial_allinone', 'interfringe': 83.58899999999998, 'peakwd': 0.9, 'scrP': [1, 2, 4, 3], 'scrB': [[2, 3], [0, 1], [1, 2], [1, 3], [0, 2], [0, 3]], 'bcd_offset': [1, 2, 3, 4, 5, 6], 'ron': 20}


...

cf shape: (7, 272, 118)
n_base: 7 n_frame: 272
Calculating piston from FFT...
cf shape: (7, 272, 23)
n_base: 7 n_frame: 272
Generating OI_WAVELENGTH...
Shape of EFF_WAVE: (118,)
Generating OI_TARGET...
Generating OI_ARRAY...
Generating OI_VIS...
Shape of complexvis: (6, 272, 23)
Shape of target_IDxxx: (1632,)
Writing OI fits...
Matching key: INSTRUME
Matching key: ESO DET CHIP NAME
Matching key: ESO DET SEQ1 DIT
Matching sky file: /Users/jscigliuto/Desktop/PDS70/MATISSE_OBS_SIPHOT_LM_SKY_077_0002.fits
loading and calibrating raw data...
Loading raw data...
Loading raw data...
Shape of bdata: (272, 118, 625)
Computing FFT of interferograms...
Shape of sum_dsp_intf: (118, 625)
Instrument {'name': 'MATISSE_L', 'ntel': 4, 'recombination': 'multiaxial_allinone', 'interfringe': 83.58899999999998, 'peakwd': 0.9, 'scrP': [1, 2, 4, 3], 'scrB': [[2, 3], [0, 1], [1, 2], [1, 3], [0, 2], [0, 3]], 'bcd_offset': [1, 2, 3, 4, 5, 6], 'ron': 20}


/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: Mean of empty slice.
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: invalid value encountered in scalar divide
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)


cf shape: (7, 272, 118)
n_base: 7 n_frame: 272
Calculating piston from FFT...
cf shape: (7, 272, 23)
n_base: 7 n_frame: 272
Generating OI_WAVELENGTH...
Shape of EFF_WAVE: (118,)
Generating OI_TARGET...
Generating OI_ARRAY...
Generating OI_VIS...
Shape of complexvis: (6, 272, 23)
Shape of target_IDxxx: (1632,)
Writing OI fits...
Matching key: INSTRUME
Matching key: ESO DET CHIP NAME
Matching key: ESO DET SEQ1 DIT
Matching sky file: /Users/jscigliuto/Desktop/PDS70/MATISSE_OBS_SIPHOT_LM_SKY_077_0002.fits
loading and calibrating raw data...
Loading raw data...
Loading raw data...
Shape of bdata: (116, 118, 625)
Computing FFT of interferograms...
Shape of sum_dsp_intf: (118, 625)
Instrument {'name': 'MATISSE_L', 'ntel': 4, 'recombination': 'multiaxial_allinone', 'interfringe': 83.58899999999998, 'peakwd': 0.9, 'scrP': [1, 2, 4, 3], 'scrB': [[2, 3], [0, 1], [1, 2], [1, 3], [0, 2], [0, 3]], 'bcd_offset': [1, 2, 3, 4, 5, 6], 'ron': 20}


/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: Mean of empty slice.
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/jscigliuto/miniconda3/envs/env_formosa/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/jscigliuto/exoMATISSE/exoMATISSE/OPTRA/op_corrflux.py:769: RuntimeWarning: invalid value encountered in scalar divide
  slope[i_base, i_frame] = np.sum((phasem-phasem.mean())*(1/wlm-np.mean(1/wlm))) / np.sum((1/wlm-np.mean(1/wlm))**2)


IndexError: index 29 is out of bounds for axis 1 with size 29